1. Tag statistics: the most frequent tags appreared;
2. Tag co-occurrence
3. 

In [1]:
import pandas as pd
import os
from collections import Counter
from itertools import combinations

In [2]:
en_path = "../Data/Reviews_Scraped/En/"
tag_file = "../Data/Tags/tags.csv"

In [3]:
# Extract tags from all the reviews and save them to a file

tags = []

header_written = False if not os.path.exists(tag_file) else True

for file in os.listdir(en_path):
    if file.endswith(".csv"):
        df = pd.read_csv(en_path + file)
        for index, row in df.iterrows():
            try:
                df.at[index, 'shelves'] = eval(row['shelves'])
            except Exception as e:
                print(f"Error in file: {file}, row: {index}")
                print(f"Row data: {row['shelves']}")
                print(f"Error message: {e}")
        
        for i in range(len(df)):
            tags.extend(df.loc[i, 'shelves'])

        with open(tag_file, 'a') as f:
            if not header_written:
                df[['shelves']].to_csv(f, index=False, header=True)
                header_written = True
            else:
                df[['shelves']].to_csv(f, index=False, header=False)

print(f"Tag information has been saved to {tag_file}")


Tag information has been saved to ../Data/Tags/tags.csv


In [4]:
print(f"Total tag numbers: {len(tags)}")
print(f"Unique tags numbers: {len(set(tags))}")

Total tag numbers: 28622
Unique tags numbers: 5684


In [5]:
# tag statistics

tag_counter = Counter(tags)
tag_counter = dict(sorted(tag_counter.items(), key=lambda item: item[1], reverse=True))
tag_counter

{'graphic-novels': 2666,
 'graphic-novel': 2011,
 'biography': 1299,
 'comics': 991,
 'non-fiction': 986,
 'history': 911,
 'nonfiction': 741,
 'memoir': 384,
 'favorites': 295,
 'graphic': 262,
 '2017': 252,
 'adult': 250,
 'historical': 218,
 'art': 213,
 'biography-memoir': 211,
 'comics-graphic-novels': 205,
 '2018': 168,
 'library': 167,
 '2020': 162,
 'graphic-novels-comics': 155,
 '2019': 132,
 'historical-fiction': 129,
 '2023': 127,
 '2022': 126,
 'young-adult': 120,
 'netgalley': 116,
 '2021': 116,
 'comics-and-graphic-novels': 109,
 'african-american': 107,
 'civil-rights': 102,
 'graphic-novels-and-comics': 99,
 '2016': 99,
 'arc': 98,
 'american-history': 87,
 'ya': 85,
 'memoirs': 82,
 'biographies': 80,
 'comic': 78,
 'autobiography': 74,
 'comic-books': 73,
 '2024': 72,
 'biographical': 71,
 'music': 70,
 'true-crime': 67,
 'own': 66,
 '2015': 64,
 'crime': 64,
 'graphics': 63,
 'library-book': 61,
 'owned': 61,
 'books-i-own': 57,
 'graphicnovels': 55,
 'reviewed': 55,

In [6]:
tag_counter_path = "../Data/Tags/tag_counter.csv"

tag_df = pd.DataFrame(list(tag_counter.items()), columns=['tag', 'frequency'])
tag_df.to_csv(tag_counter_path, index=False)

In [ ]:
# Tag co-occurrence matrix

tagsAll = pd.read_csv(tag_file)
tagsAll = tagsAll['shelves'].apply(eval)
tagsAll = tagsAll.tolist()

cooccur = {}

for tagList in tagsAll:
    if len(tagList) < 2:
        continue
    
    # calculate tag co-occurrence with combination
    for pair in combinations(tagList, 2):
        if pair in cooccur:
            cooccur[pair] += 1
        else:
            cooccur[pair] = 1

tagSet = list(tag_counter.keys()) # get unique tags with sorted in frequency
coMatrix = pd.DataFrame(index=tagSet, columns=tagSet, data=0)

for (tag1, tag2), freq in cooccur.items():
    coMatrix.at[tag1, tag2] = freq
    coMatrix.at[tag2, tag1] = freq

coMatrix.iloc[:10, :10]

In [8]:
coMatrix_path = "../Data/Tags/tag_coMatrix.csv"
coMatrix.to_csv(coMatrix_path)

In [11]:
tagSet[:279]

['graphic-novels',
 'graphic-novel',
 'biography',
 'comics',
 'non-fiction',
 'history',
 'nonfiction',
 'memoir',
 'favorites',
 'graphic',
 '2017',
 'adult',
 'historical',
 'art',
 'biography-memoir',
 'comics-graphic-novels',
 '2018',
 'library',
 '2020',
 'graphic-novels-comics',
 '2019',
 'historical-fiction',
 '2023',
 '2022',
 'young-adult',
 'netgalley',
 '2021',
 'comics-and-graphic-novels',
 'african-american',
 'civil-rights',
 'graphic-novels-and-comics',
 '2016',
 'arc',
 'american-history',
 'ya',
 'memoirs',
 'biographies',
 'comic',
 'autobiography',
 'comic-books',
 '2024',
 'biographical',
 'music',
 'true-crime',
 'own',
 '2015',
 'crime',
 'graphics',
 'library-book',
 'owned',
 'books-i-own',
 'graphicnovels',
 'reviewed',
 'memoir-biography',
 'science',
 '2012',
 'digital',
 '2014',
 'graphic-novel-comic',
 'biography-and-memoir',
 'gn-bio',
 '2013',
 'book-club',
 'comic-graphic-novel',
 'library-books',
 'fiction',
 'comix',
 'horror',
 'graphic-memoir',
 'fa

In [12]:
# Tag co-occurrence matrix with tags appeared at least 10 times

coMatrixShort = coMatrix.loc[tagSet[:279], tagSet[:279]]
coMatrixShort_path = "../Data/Tags/tag_coMatrix_short.csv"
coMatrixShort.to_csv(coMatrixShort_path)

In [14]:
gephiNodeFile = "../Data/Tags/tag_gephi_id_freq.csv"
gephiNode = pd.read_csv(gephiNodeFile)

gephiNode.insert(1, 'Label', gephiNode['Id'])

gephiNode.to_csv(gephiNodeFile, index=False, mode='w')